## Takeaways
- 11/20/2022
    - Clean up S&P 500 data frame before it gets read into a CSV: lots of empty columns and also an empty row at the top
    - Create a column that tracks dividend growth history
    

## App Testing

In [85]:
# Import Libraries
import pandas as pd
import yfinance as yf
import streamlit as st
import os
import datetime
from pandas.tseries.offsets import BDay

pwd = os.getcwd()

In [2]:
import pandas as pd
import yfinance as yf
import datetime 
from pandas.tseries.offsets import BDay

def ticker_info(): # ticker_symbol

    l = ["MCD", "PEP", "MSFT", "AAPL", "O", "PG", "JNJ"]

    df_list = []

    for t in l:
        df_list.append(pd.DataFrame([yf.Ticker(t).info]))
    
    df = pd.concat(df_list)

    df["Current_Date"] = datetime.datetime.today() - BDay(0)
    df["Previous_Date"] = datetime.datetime.today() - BDay(1)
    df["Last_Dividend_Date"] = df["lastDividendDate"]

    return df

# df = ticker_info()

In [125]:

df["Last_Dividend_Date"] = pd.to_datetime(df["Last_Dividend_Date"])
df["Last_Dividend_Date"]

0   1970-01-01 00:00:01.661904000
0   1970-01-01 00:00:01.661990400
0   1970-01-01 00:00:01.660694400
0   1970-01-01 00:00:01.667520000
0   1970-01-01 00:00:01.667174400
0   1970-01-01 00:00:01.666224000
0   1970-01-01 00:00:01.661126400
Name: Last_Dividend_Date, dtype: datetime64[ns]

In [8]:
def ticker_history(ticker_symbol):

    # Call yfinance ticker info
    ticker = yf.Ticker(ticker_symbol)

    # Assign variable for ticker history
    ticker_history = ticker.history(period="max")

    return ticker_history


ticker_history("MCD", "MSFT")

TypeError: ticker_history() takes 1 positional argument but 2 were given

## Execute staging data

In [ ]:
# Assign function call to ticker
mcd_ticker = ticker_price_action("MCD")
pep_ticker = ticker_price_action("PEP")
msft_ticker = ticker_price_action("MSFT")
o_ticker = ticker_price_action("O")

# Export to CSV
mcd_ticker.to_csv(pwd + "\\mcd_stocks.csv", mode ="w", header=True, index=False)
pep_ticker.to_csv(pwd + "\\pep_stocks.csv", mode ="w", header=True, index=False)
msft_ticker.to_csv(pwd + "\\msft_stocks.csv", mode ="w", header=True, index=False)
o_ticker.to_csv(pwd + "\\o_stocks.csv", mode ="w", header=True, index=False)

# Create CSV data frame the import into Streamlit
mcd_df = pd.read_csv(pwd + "\\mcd_stocks.csv")
pep_df = pd.read_csv(pwd + "\\pep_stocks.csv")
msft_df = pd.read_csv(pwd + "\\msft_stocks.csv")
o_df = pd.read_csv(pwd + "\\o_stocks.csv")

## Previous script versions

### Iteration 2: Consolidated into functions, but very high coupling and low cohesion (bad)

In [5]:
def ticker_function(ticker_symbol):

    # Call yfinance ticker info
    ticker = yf.Ticker(ticker_symbol)

    # Assign variable for ticker history
    ticker_history = ticker.history(period="max")

    # Number of rows displayed
    rows_returned = 5

    # Create derived columns
    ticker_history["Ticker"] = ticker_symbol
    ticker_history["Wkly_Avg_Close"] = ticker_history["Close"].rolling(7).mean()
    ticker_history["Wkly_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Wkly_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Monthly_Avg_Close"] = ticker_history["Close"].rolling(30).mean()
    ticker_history["Monthly_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Monthly_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Semi_Annual_Avg_Close"] = ticker_history["Close"].rolling(183).mean()
    ticker_history["Semi_Annual_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Semi_Annual_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Annual_Avg_Close"] = ticker_history["Close"].rolling(365).mean()
    ticker_history["Annual_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Annual_Avg_Close"]) / ticker_history["Close"] * 100
    ticker_history["Two_Year_Avg_Close"] = ticker_history["Close"].rolling(730).mean()
    ticker_history["Two_Year_Avg_Close_Percent"] = (ticker_history["Close"] - ticker_history["Two_Year_Avg_Close"]) / ticker_history["Close"] * 100

    # Drop NAs
    ticker_history = ticker_history.dropna()

    # Create new DF of only needed columns
    ticker_price_action = ticker_history[["Ticker", "Close", 
                                    "Wkly_Avg_Close_Percent", "Monthly_Avg_Close_Percent",
                                    "Semi_Annual_Avg_Close_Percent", "Annual_Avg_Close_Percent", "Two_Year_Avg_Close_Percent",
                                    "Wkly_Avg_Close", "Monthly_Avg_Close", 
                                    "Semi_Annual_Avg_Close", "Annual_Avg_Close", "Two_Year_Avg_Close"
                                    ]] \
                                    .sort_values(by=["Date"], ascending=False) \
                                    .head(rows_returned)
    ticker_price_action = ticker_price_action.reset_index()

    ticker_price_action["Date"] = ticker_price_action["Date"].dt.date

    return ticker_price_action

# Assign function call to ticker
# mcd_ticker = ticker_function("MCD")
# pep_ticker = ticker_function("PEP")
# msft_ticker = ticker_function("MSFT")

# Export to CSV
# mcd_ticker.to_csv(pwd + "\\mcd_stocks.csv", mode ="w", header=True, index=False)
# pep_ticker.to_csv(pwd + "\\pep_stocks.csv", mode ="w", header=True, index=False)
# msft_ticker.to_csv(pwd + "\\msft_stocks.csv", mode ="w", header=True, index=False)

# Create CSV data frame the import into Streamlit
# df = pd.read_csv(pwd + "\\stocks.csv")

# Run Streamlit
# st.title("Weekly DCA Assessment")
# st.write(df)

# add_sidebar = st.sidebar.selectbox("Test Side Bar Title", ("Test 1", "Test 2"))



### Iteration 1: Not yet consolidated into functions

In [105]:
mcd = yf.Ticker("mcd")
mcd_history = mcd.history(period="max")

# Create Ticker Column
mcd_history["Ticker"] = "MCD"

# Create rolling weekly average column
mcd_history["weekly_avg_close"] = mcd_history["Close"].rolling(7).mean()

# Create weekly average close percent column
mcd_history["weekly_avg_close_percent"] = (mcd_history["Close"] - mcd_history["weekly_avg_close"]) / mcd_history["Close"] * 100

# Create rolling monthly average column
mcd_history["monthly_avg_close"] = mcd_history["Close"].rolling(30).mean()

# Create monthly average close percent column
mcd_history["monthly_avg_close_percent"] = (mcd_history["Close"] - mcd_history["monthly_avg_close"]) / mcd_history["Close"] * 100

# Create rolling semi_annual average column
mcd_history["semi_annual_avg_close"] = mcd_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
mcd_history["semi_annual_avg_close_percent"] = (mcd_history["Close"] - mcd_history["semi_annual_avg_close"]) / mcd_history["Close"] * 100

# Create rolling annual average column
mcd_history["annual_avg_close"] = mcd_history["Close"].rolling(365).mean()

# Create annual average close percent column
mcd_history["annual_avg_close_percent"] = (mcd_history["Close"] - mcd_history["annual_avg_close"]) / mcd_history["Close"] * 100


# Create rolling two_year average column
mcd_history["two_year_avg_close"] = mcd_history["Close"].rolling(730).mean()

# Create two_year average close percent column
mcd_history["two_year_avg_close_percent"] = (mcd_history["Close"] - mcd_history["two_year_avg_close"]) / mcd_history["Close"] * 100

# Drop NAs
mcd_history = mcd_history.dropna()

# Grab only needed columns
mcd_price_action = mcd_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
mcd_price_action = mcd_price_action.reset_index()

# Export to CSV
# mcd_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)

In [106]:
pep = yf.Ticker("PEP")
pep_history = pep.history(period="max")

# Create Ticker Column
pep_history["Ticker"] = "PEP"

# Create rolling weekly average column
pep_history["weekly_avg_close"] = pep_history["Close"].rolling(7).mean()

# Create weekly average close percent column
pep_history["weekly_avg_close_percent"] = (pep_history["Close"] - pep_history["weekly_avg_close"]) / pep_history["Close"] * 100

# Create rolling monthly average column
pep_history["monthly_avg_close"] = pep_history["Close"].rolling(30).mean()

# Create monthly average close percent column
pep_history["monthly_avg_close_percent"] = (pep_history["Close"] - pep_history["monthly_avg_close"]) / pep_history["Close"] * 100

# Create rolling semi_annual average column
pep_history["semi_annual_avg_close"] = pep_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
pep_history["semi_annual_avg_close_percent"] = (pep_history["Close"] - pep_history["semi_annual_avg_close"]) / pep_history["Close"] * 100

# Create rolling annual average column
pep_history["annual_avg_close"] = pep_history["Close"].rolling(365).mean()

# Create annual average close percent column
pep_history["annual_avg_close_percent"] = (pep_history["Close"] - pep_history["annual_avg_close"]) / pep_history["Close"] * 100

# Create rolling two_year average column
pep_history["two_year_avg_close"] = pep_history["Close"].rolling(730).mean()

# Create two_year average close percent column
pep_history["two_year_avg_close_percent"] = (pep_history["Close"] - pep_history["two_year_avg_close"]) / pep_history["Close"] * 100


# Drop NAs
pep_history = pep_history.dropna()

# Grab only needed columns
pep_price_action = pep_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
pep_price_action = pep_price_action.reset_index()


# Export to CSV
# pep_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)

In [107]:
msft = yf.Ticker("MSFT")
msft_history = msft.history(period="max")

msft_history["Ticker"] = "MSFT"

# Create rolling weekly average column
msft_history["weekly_avg_close"] = msft_history["Close"].rolling(7).mean()

# Create weekly average close percent column
msft_history["weekly_avg_close_percent"] = (msft_history["Close"] - msft_history["weekly_avg_close"]) / msft_history["Close"] * 100

# Create rolling monthly average column
msft_history["monthly_avg_close"] = msft_history["Close"].rolling(30).mean()

# Create monthly average close percent column
msft_history["monthly_avg_close_percent"] = (msft_history["Close"] - msft_history["monthly_avg_close"]) / msft_history["Close"] * 100

# Create rolling semi_annual average column
msft_history["semi_annual_avg_close"] = msft_history["Close"].rolling(183).mean()

# Create semi_annual average close percent column
msft_history["semi_annual_avg_close_percent"] = (msft_history["Close"] - msft_history["semi_annual_avg_close"]) / msft_history["Close"] * 100

# Create rolling annual average column
msft_history["annual_avg_close"] = msft_history["Close"].rolling(365).mean()

# Create annual average close percent column
msft_history["annual_avg_close_percent"] = (msft_history["Close"] - msft_history["annual_avg_close"]) / msft_history["Close"] * 100

# Create rolling two_year average column
msft_history["two_year_avg_close"] = msft_history["Close"].rolling(730).mean()

# Create two_year average close percent column
msft_history["two_year_avg_close_percent"] = (msft_history["Close"] - msft_history["two_year_avg_close"]) / msft_history["Close"] * 100


# Drop NAs
msft_history = msft_history.dropna()

# Grab only needed columns
msft_price_action = msft_history[["Ticker", "Close", 
                                "weekly_avg_close_percent", "monthly_avg_close_percent",
                                "semi_annual_avg_close_percent", "annual_avg_close_percent", "two_year_avg_close_percent",
                                "weekly_avg_close", "monthly_avg_close", 
                                "semi_annual_avg_close", "annual_avg_close", "two_year_avg_close"
                                ]] \
                                .sort_values(by=["Date"], ascending=False) \
                                .head(5)
msft_price_action = msft_price_action.reset_index()

# Export to CSV
# msft_price_action.to_csv(pwd + "\\stocks.csv", mode ="a", header=True, index=False)